# Objective

Help a nonprofit organization YoLocal Snack find a location to open up a store. Their goal is to provide nourishing local snacks and drinks to daily commuters during common meal hours. To make the most impact, YoLocal Snack's ideal store location is by a subway station in a local community with a large population of New Yorkers with long weekday commute times.  

#### Common Meal Hours
- 8am-12pm = breakfast 
- 12pm-4pm = lunch
- 4pm-8pm = happy hour

### Find the High Traffic Commuter Stations

We want high entry numbers in the morning and high exit numbers in the evening, and for the morning entry traffic and evening traffic to be similar by the end of the day. This indicates a strong population of 9 to 5 workers and students residing near a particular station.

```Note: Within these commuter stations are outliers with very high traffic because New Yorkers travel to certain stations to exit for work or transfer to another form of transportation. All of these stations are in Manhattan and will be excluded from our list of commuter stations because it is difficult to determine if the commuters in Manhattan are traveling a long distance. We can use this data to show where commuters are in certain points of day.```

### Establishing Long Distance Commuters

The NYC MTA Subway map reveals that the majority of subway stations in boroughs outside of Manhattan contain only one or two lines. These stations serve different neighborhoods and the distance between each line and Manhattan establishes a clear distinction of a long commute.  

### Further checks

After identifying potential stations, we can do a google search of the station to see how many local snack stores are nearby. Google's activity tracker reveals if the stores see an increase in traffic during meal hours. In the future, YoLocal Snack will work with these vendors to efficiently cater to local commuters. 

# Gathering Data

MTA data from January 2021 to April 2021 will form the basis of my analysis. This is a good time frame to look at New York's commuter cycle. Students go back to school in January and workers resume work after major holidays. Additionally, the turnstile data has reset so it's possible to detect where anomalies begin and decide what to do with them. Decreases in commuter traffic due to COVID is not a concern because YoLocal Snack is serving New Yorkers who need to commute to work.


### MTA Turnstile Data 

##### Purpose: 

- Entry and Exit Traffic Numbers
- Stations and Line Names
- Date and Time  

### MTA Location Data

##### Purpose: 

- Borough
- Latitude and Longitude

### MTA RIDERSHIP DATA

##### Purpose: 

- The total ridership from January 1, 2021 to April 23, 2021 is [171,715,108](https://new.mta.info/document/20441)
- Estimate daily ridership

In [ ]:
from sqlalchemy import create_engine
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import calendar
import datetime 
from datetime import timedelta
%config InlineBackend.figure_format = 'svg'
%matplotlib inline 

In [ ]:
pd.set_option('display.max.colwidth', None)

In [ ]:
daily_total_riders_url = "https://new.mta.info/document/20441"

# Loading the Data

Make a daily total dataframe by loading the csv with pandas. 

Access the mta database with sql alchemy and left join the turnstile table with location data by Stop_Name and Division. Then create a dataframe with the joined tables.

- " WITH turnstile21_data as 
- (SELECT * FROM turnstile_data WHERE DATE LIKE '%2021')
- SELECT t.*, l.Borough, l.GTFS_Latitude, l.GTFS_Longitude
- FROM turnstile21_data t LEFT JOIN location_data l
- ON t.STATION = upper(l.Stop_Name) AND t.DIVISION = l.Division;"

The top code works, but takes a long time to load on dbrowser so I will not use it on jupyter notebook. We'll have to import them individually and join them through pandas. 

In [ ]:
daily_total_raw_df = pd.read_csv(daily_total_riders_url)
daily_total_raw_df.rename(columns = {'Date': 'DATE', 'Subways: Total Estimated Ridership':'SUBWAY_TOTAL'}, inplace = True)
daily_total_df = daily_total_raw_df.loc[:, ['DATE','SUBWAY_TOTAL']]
daily_total_df['YEAR'] = daily_total_df['DATE'].str.extract(r'\b(\d+)$')
daily_total_21_df = daily_total_df[daily_total_df['YEAR'] == '2021']

In [ ]:
engine = create_engine("sqlite:///Data/mta.db")

In [ ]:
turnstile_df_21 = pd.read_sql("SELECT * FROM turnstile_data WHERE DATE LIKE '%2021';", engine)

In [ ]:
turnstile_df_21.columns = turnstile_df_21.columns.str.replace(' ','')

In [ ]:
location_df = pd.read_sql("SELECT Stop_Name, Division, Borough, GTFS_Latitude, GTFS_Longitude FROM location_data;", engine)

In [ ]:
location_df['Stop_Name'] = location_df['Stop_Name'].str.upper()

In [ ]:
turn_loc_21_df = turnstile_df_21.merge(location_df, how = 'left', left_on=['STATION'], right_on=['Stop_Name'])

In [ ]:
turn_loc_21_df.info()

In [ ]:
turn_loc_21_df.describe()

In [ ]:
turn_loc_21_df['ENTRIES'] = turn_loc_21_df['ENTRIES'].astype('int')
turn_loc_21_df['EXITS'] = turn_loc_21_df['EXITS'].astype('int')

In [ ]:
turn_loc_21_df['ENTRIES'].describe()

In [ ]:
turn_loc_21_df.head()

# DATA CLEANING Part 1

A quick exploration of the dataset reveals many cleaning tasks. There are duplicate rows, the exits and entries columns contain outliers that are far from the mean, the time column reveals 62524 instead of the expected 7. The entries and exits columns show cumulative values instead of the number of entries at that point in time. 

The next steps will include:
1. Reformat the unique 62524 time values to the standard 
2. Combine DATE and TIME and create an indentification for a unique turnstile
3. Create a Day column and weekday mask  
4. Locate the outliers greater than a hundred million because there were only 171,715,108 total riderships across stations in this time period.  
5. Removing the duplicate values 
6. Calculate the number of entries and exits for each turnstile  

In [ ]:
temp_time = turn_loc_21_df['TIME'].reset_index().copy()

In [ ]:
temp_time['first3'] = temp_time['TIME'].str[:4]
temp_time['first3'] = temp_time['first3'].str.replace(":", ".").astype('float')
temp_time['first3'] = temp_time['first3'].apply(lambda x: np.round(x,0))

In [ ]:
time_bin = [0.0, 4.0, 8.0, 12.0, 16.0, 20.0, 24.0]
temp_time['first3'] = temp_time['first3'].apply(lambda x: time_bin[np.digitize(x,time_bin, right = True)])

In [ ]:
temp_time['first3'] = temp_time['first3'].astype('string')

In [ ]:
time_dict = {'0.0': '00:00:00' , '4.0': '04:00:00' , '8.0': '08:00:00' , '12.0': '12:00:00', 
             '16.0': '16:00:00', '20.0': '20:00:00', '24.0': '00:00:00' }
temp_time['first3'] = temp_time['first3'].map(time_dict)

In [ ]:
time_list = list(temp_time['first3'])
turn_loc_21_df['TIME'] = time_list

In [ ]:
turn_loc_21_df['TIME'].nunique()

In [ ]:
mta_dfs = [turn_loc_21_df]

for mta_df in mta_dfs:
    
    mta_df['DATETIME'] = pd.to_datetime(mta_df.DATE + " " + mta_df.TIME, 
                                        format="%m/%d/%Y %H:%M:%S")
    
    mta_df['TURNSTILES'] = mta_df['C/A'] + "-" +\
                           mta_df['UNIT'] + "-" +\
                           mta_df['SCP'] + "-" +\
                           mta_df['STATION'] 

In [ ]:
day_of_week_dict = dict(enumerate(calendar.day_name))
turn_loc_21_df['DAYNAME'] = turn_loc_21_df['DATETIME'].dt.weekday.map(day_of_week_dict)

In [ ]:
weekday_mask = (turn_loc_21_df['DAYNAME'].isin(calendar.day_name[:5]))

In [ ]:
turn_loc_21_df = turn_loc_21_df[['TURNSTILES', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 
                                   'BOROUGH', 'GTFS_LATITUDE', 'GTFS_LONGITUDE', 'DATETIME', 
                                   'DATE', 'DAYNAME', 'TIME','ENTRIES', 'EXITS']]

In [ ]:
(turn_loc_21_df.groupby(['TURNSTILES','DATETIME'])
['ENTRIES', 'EXITS'].count()
.reset_index()
.sort_values(["ENTRIES", "EXITS"], ascending=False)).head()

In [ ]:
turn_loc_21_df.sort_values(['TURNSTILES','DATETIME'], 
                   ascending = True, inplace = True)
turn_loc_21_df.drop_duplicates(subset = ['TURNSTILES', 'DATETIME'], keep = 'first',
                      inplace = True)

In [ ]:
turn_loc_21_df[["PREV_DATE", "PREV_ENTRIES", "PREV_EXITS"]] = (turnstile_df_21
                                                       .groupby(["TURNSTILES"])["DATE", "ENTRIES", "EXITS"]
                                                       .apply(lambda grp: grp.shift(1)))
turn_loc_21_df.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [ ]:
total_traffic_21 = daily_total_21_df.groupby('YEAR')['SUBWAY_TOTAL'].sum()['2021']
print(total_traffic_21)
print(len(str(total_traffic_21)))

In [ ]:
turn_loc_21_df['IRR_ENTRIES']=turn_loc_21_df['ENTRIES'].apply(lambda x: len(str(x))>=8) 
turn_loc_21_df['IRR_EXITS']=turn_loc_21_df['EXITS'].apply(lambda x: len(str(x))>=8) 

In [ ]:
clean_turnstile_df_21 = turn_loc_21_df[~(turn_loc_21_df['irr_entry'] == True) & 
                                        ~(turn_loc_21_df['irr_exit'] == True)]

In [ ]:
clean_turnstile_df_21.shape

# DATA CLEANING Part 2

We reformatted the dates to fall into date ranges 0-4,4-8,8-12,4-8,8-12 to make it easier for analysis. The outliers that fall outside of the total number of traffic in January to April 2021 have been masked and removed. 

Before we calculate the entries and exits for a particular point in time, we need to perform a gutcheck. The ideal stiatuion is to have all the previous entries be less than the current entries. We want to check if there are situations where PREV_ENTRIES > ENTRIES or PREV_EXITS > EXITS and then decide how to calculate the entries and exits. We can sort the entries and exits descending order with a groupby. 

In [ ]:
mask = (turnstile_df_21["ENTRIES"] < turnstile_df_21["PREV_ENTRIES"])
turnstile_df_21[mask].groupby(["TURNSTILES"]).size().sort_values(ascending = False).head(20)

In [ ]:
clean_mask = (clean_turnstile_df_21["ENTRIES"] < clean_turnstile_df_21["PREV_ENTRIES"])
clean_turnstile_df_21[mask].groupby(["TURNSTILES"]).size().sort_values(ascending = False).head(20)

In [ ]:
max_counter = 4 * 3600 * 2

In [ ]:
def entries_diff(row, max_counter):
    counter = abs(row['ENTRIES'] - row['PREV_ENTRIES'])
    if counter > max_counter:
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        return 0
    return counter
def exits_diff(row, max_counter):
    counter = abs(row['EXITS'] - row['PREV_EXITS'])
    if counter > max_counter:
        counter = min(row["EXITS"], row["PREV_EXITS"])
    if counter > max_counter:
        return 0
    return counter

In [ ]:
clean_turnstile_df_21['ENTRIES_COUNT'] = clean_turnstile_df_21.apply(entries_diff, axis = 1, max_counter = max_counter)

In [ ]:
clean_turnstile_df_21['ENTRIES_COUNT'].sort_values(ascending = False)

In [ ]:
clean_turnstile_df_21['EXITS_COUNT'] = clean_turnstile_df_21.apply(exits_diff, axis = 1, max_counter = max_counter)

In [ ]:
clean_turnstile_df_21['TOTAL_COUNT'] = clean_turnstile_df_21['ENTRIES_COUNT'] + clean_turnstile_df_21['EXITS_COUNT']

# DATA ANALYSIS WITH ONLY TURNSTILE DATA

After finding the entries and exits values, we can combine the two values to find the total traffic for a particular turnstile at a time in day. 

Questions:

1. Find the top 20 stations with the highest number of exits, enteries, traffic
    - Now find the top stations with only one or two lines with the highest number of exits, entries, traffic
2. Using the results from question one, we find the stations with highest exits, entries, traffics for time ranges 8-12, 12-4, 4-8 *meal hours
    
    - Which stations have the most entries around 8-12 am
    - Which stations have the most exits around 4-8pm pm
 
    
    - Which stations have the most exits around 8-12 am?
    - Which stations have the most entriess around 4 - 8 pm? 
    
3. Find the average total of exits, entries, traffic for each weekday
    - Do entries = exits?
    - Is traffic consistent throughout the weekdays
    - Using total traffic establish percentage of people in certain stations?


In [ ]:
top_20_entries = clean_turnstile_df_21.groupby('STATION')['ENTRIES_COUNT'].sum().head(20).sort_values(ascending = False)

In [ ]:
clean_turnstile_df_21[weekday_mask]

In [ ]:
top_20_exits = clean_turnstile_df_21.groupby('STATION')['EXITS_COUNT'].sum().head(20).sort_values(ascending = False)

In [ ]:
top_20_traffic = clean_turnstile_df_21.groupby('STATION')['TOTAL_COUNT'].sum().head(20).sort_values(ascending = False)

In [ ]:
mask = (clean_turnstile_df_21['LINENAME'].str.len() == 2)
two_or_one_line_df = clean_turnstile_df_21[mask].copy()

In [ ]:
top_20_entries_2 = two_or_one_line_df.groupby('STATION')['ENTRIES_COUNT'].sum().head(20).sort_values(ascending = False)
top_20_entries_2

In [ ]:
top_20_exits_2 = two_or_one_line_df.groupby('STATION')['EXITS_COUNT'].sum().head(20).sort_values(ascending = False)
top_20_exits_2

In [ ]:
top_20_traffic_2 = two_or_one_line_df.groupby('STATION')['TOTAL_COUNT'].sum().head(20).sort_values(ascending = False)
top_20_traffic_2

In [ ]:
daily_total_check_df_21

# DATA VIZUALIZATIONS WITH ONLY TURNSTILE DATA

Plot the answers to the questions to itentify insights and potential gaps in data.

- Bar Chart -> Top 20 stations highest exits, entries, traffic 
- Line chart -> Consistency of Entries and Exits over time for a station (We're looking for consistent traffic)
- Scatter Plot -> Exits versus Entries for a particular station 
- Heatmat -> Traffic flow during the weekday by TIME of a particular station

# ADDING FARE AND LOCATION DATASETS

# CONCLUSION

# FUTURE IDEAS

1. Another way to estimate the number of daily regular schedule commuters at a station is to find the total number of unlimited and student metro cards used daily.